In [1]:
import pickle
import datetime
import torch
import numpy as np
from scipy import stats
import joblib
from scipy.sparse import csr_matrix
from itertools import chain, combinations
from sklearn.linear_model import LogisticRegression

Для начала посмотрим что представляют собой данные и переформатируем их, выкинув лишнее, чтобы не нагружать RAM (при первых попытках решения задачи доступного объёма опреативной памяти не хватило).

In [2]:
def read_pickle(filename):
    objects = []
    with open(filename, "rb") as openfile:
        try:
            return pickle.load(openfile)
        except EOFError:
            return None
    return None

In [13]:
players = read_pickle('players.pkl')
results = read_pickle('results.pkl')
tournaments = read_pickle('tournaments.pkl')

In [3]:
def dump_to_pickle(obj, filename):
    with open(filename, 'wb') as pickle_file:
        pickle.dump(obj, pickle_file)

Из информации об игроках оставим только player_id и полное имя.

In [9]:
player_id_to_name = {}
for _, player in players.items():
    player_id_to_name[player['id']] = ('' if player['surname'] is None else (player['surname'] + ' ')) + ('' if player['name'] is None else (player['name'] + ' ')) + ('' if player['patronymic'] is None else player['patronymic'])

In [10]:
player_id_to_name

{1: 'Абабилов Алексей ',
 10: 'Абалов Игорь ',
 11: 'Абалымова Наталья Юрьевна',
 12: 'Абальян Артур Евгеньевич',
 13: 'Абальян Эрик Евгеньевич',
 14: 'Абанин Василий ',
 15: 'Абарников Олег Игоревич',
 16: 'Абасалиев Азер Абасали оглы',
 17: 'Абасев А. В.',
 18: 'Аббасханов Гияс ',
 19: 'Абашев Александр Миронович',
 20: 'Абашин Владимир Васильевич',
 21: 'Абашин Павел Николаевич',
 22: 'Абашин Павел ',
 23: 'Абащенко Андрей Николаевич',
 25: 'Аббасов Азер ',
 26: 'Аббасов Алексей ',
 27: 'Аббасов Гашам ',
 29: 'Аббасов Низами Азерович',
 30: 'Аббасов Орхан Вахид оглы',
 31: 'Абасова Ситара Фахраддин гызы',
 33: 'Аббясов Аник ',
 34: 'Аббясова Венера ',
 35: 'Абгарян Нарек Гагикович',
 36: 'Абдалла Хасан-Ахмад-Амин ',
 37: 'Абдалов Егор ',
 38: 'Абдеев Эдуард ',
 39: 'Авдеева Яна ',
 40: 'Абдикалыков Абдикожа ',
 41: 'Абдрахманов Айдар Ильгизарович',
 42: 'Абдрахманов Андрей ',
 43: 'Абдрахманов Артур ',
 44: 'Абдрахманов Радик ',
 45: 'Абдрахманов Ринат Ильдарович',
 46: 'Абдрахманов

In [11]:
dump_to_pickle(player_id_to_name, 'player_id_to_name.pkl')

In [12]:
tournaments[14]

{'id': 14,
 'name': 'Самариум',
 'dateStart': '2003-11-08T00:00:00+03:00',
 'dateEnd': '2003-11-09T00:00:00+03:00',
 'type': {'id': 2, 'name': 'Обычный'},
 'season': '/seasons/2',
 'orgcommittee': [],
 'synchData': None,
 'questionQty': None}

Из информации о турнирах нам может понадобится id, название и тип ("test" - проходил в 2020 году, "train" - проходил в 2019 году, "other" - все остальные)

In [14]:
def get_tournament_year(tournament):
    return int(tournament['dateStart'].split('T')[0].split('-')[0])

In [15]:
tournaments_processed = {}
for id, tournament in tournaments.items():
    year = get_tournament_year(tournament)
    if year == 2020:
        t = 'test'
    elif year == 2019:
        t = 'train'
    else:
        t = 'other'
    tournaments_processed[tournament['id']] = {
        'id': tournament['id'],
        'name': tournament['name'],
        'type': t
    }

In [15]:
tournaments_processed

{1: {'id': 1, 'name': 'Чемпионат Южного Кавказа', 'test': False},
 2: {'id': 2, 'name': 'Летние зори', 'test': False},
 3: {'id': 3, 'name': 'Турнир в Ижевске', 'test': False},
 4: {'id': 4, 'name': 'Чемпионат Украины. Переходной этап', 'test': False},
 5: {'id': 5, 'name': 'Бостонское чаепитие', 'test': False},
 6: {'id': 6, 'name': 'Нестерка', 'test': False},
 7: {'id': 7, 'name': 'Фестиваль, посвящённый 60-летию ЮрГУ', 'test': False},
 8: {'id': 8, 'name': 'Чемпионат мира', 'test': False},
 9: {'id': 9, 'name': 'Неспростая зима', 'test': False},
 10: {'id': 10, 'name': 'Гостиный двор', 'test': False},
 11: {'id': 11, 'name': 'Белые ночи - Инжэкон', 'test': False},
 12: {'id': 12, 'name': 'Кубок провинций', 'test': False},
 13: {'id': 13, 'name': 'Летний лагерь в Баварии (Хоббах)', 'test': False},
 14: {'id': 14, 'name': 'Самариум', 'test': False},
 15: {'id': 15,
  'name': 'Открытый Всероссийский синхронный чемпионат',
  'test': False},
 16: {'id': 16, 'name': 'Открытый кубок России

In [19]:
dump_to_pickle(tournaments_processed, 'tournaments_processed.pkl')

In [17]:
results[1]

[{'team': {'id': 242,
   'name': 'Команда Азимова',
   'town': {'id': 21, 'name': 'Баку'}},
  'mask': None,
  'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}},
  'questionsTotal': 0,
  'synchRequest': None,
  'position': 1,
  'controversials': [],
  'flags': [],
  'teamMembers': [{'flag': None,
    'usedRating': 0,
    'rating': 0,
    'player': {'id': 476,
     'name': 'Анар',
     'patronymic': 'Беюкага оглы',
     'surname': 'Азимов'}},
   {'flag': None,
    'usedRating': 0,
    'rating': 0,
    'player': {'id': 878,
     'name': 'Фариз',
     'patronymic': 'Наим оглы',
     'surname': 'Аликишибеков'}},
   {'flag': None,
    'usedRating': 0,
    'rating': 0,
    'player': {'id': 1872,
     'name': 'Аднан',
     'patronymic': 'Фариз оглы',
     'surname': 'Ахундов'}},
   {'flag': None,
    'usedRating': 0,
    'rating': 0,
    'player': {'id': 13721,
     'name': 'Балаш',
     'patronymic': 'Алекпер оглы',
     'surname': 'Касумов'}},
   {'flag': None,
    '

В результатах также оставим только нужные данные. Удалим из результатов турнира те команды, для которых имеются не все повопросные результаты, а также команды, для которых не указан список участников.

In [16]:
results_processed = []
for tournament_id, result in results.items():
    team_results = []
    for team_result in result:
        max_mask_len = 0
        if 'mask' in team_result and team_result['mask'] is not None and 'X' not in team_result['mask'] and '?' not in team_result['mask']:
            max_mask_len = max(max_mask_len, len(team_result['mask']))
    for team_result in result:
        if 'mask' in team_result and team_result['mask'] is not None and len(team_result['mask']) == max_mask_len and 'X' not in team_result['mask'] and '?' not in team_result['mask']:
            team_result_processed = {}
            team_result_processed['mask'] = team_result['mask']
            if 'position' in team_result:
                team_result_processed['position'] = team_result['position']
            team_members = []
            for team_member in team_result['teamMembers']:
                team_members.append(team_member['player']['id'])
            if len(team_members):
                team_result_processed['team_members'] = team_members
                team_results.append(team_result_processed)
    if len(team_results):
        result_processed = {
            'tournament_id': tournament_id,
            'team_results': team_results,
            'type': tournaments_processed[tournament_id]['type']
        }
        results_processed.append(result_processed)

In [39]:
results_processed[:10]

[{'tournament_id': 22,
  'team_results': [{'mask': '011101110110111000110111001111111111001111110001111111111111111111110101111100011000111111',
    'position': 1,
    'team_members': [1560, 2935, 3270, 4878, 18935, 32979, 36497]},
   {'mask': '011111110101101001010111011111111111001101111101011101111101111110110101111100100001111111',
    'position': 2.5,
    'team_members': [707, 13551, 15442, 25882, 30475, 34846]},
   {'mask': '001111110101101001110111001111111111011111111101011011011011111111110000111100011011111011',
    'position': 2.5,
    'team_members': [2694, 20691, 22482, 22935, 28513, 29800]},
   {'mask': '011111100110111000110111101111111111010011111101111011101011101110110011111100011011111001',
    'position': 4.5,
    'team_members': [5526, 14259, 18686, 20260, 21805, 29333]},
   {'mask': '011101100101111100110111101111110101011111110101011011111011101110010111011100001001111111',
    'position': 6.5,
    'team_members': [3207, 3645, 6212, 8333, 49151]},
   {'mask': '01

In [17]:
dump_to_pickle(results_processed, 'results_processed.pkl')

В качестве baseline модели возьмем бинарный классификатор (логистическую регрессию), обученный на парах игрок-вопрос, предсказывающий вероятность того, что игрок ответил на вопрос. На вход модели будем подавать вектор длины, равной сумме количества игроков и количества вопросов, в котором элемент, соответствующий игроку равен 1, а элемент, соответствующий вопросу, равен -1. Таким образом на выходе линейного слоя модели мы получим разницу силы игрока и сложности вопроса. Если она больше 0, то игрок вероятнее всего ответил на вопрос, а если меньше, то нет.

In [4]:
player_id_to_name = read_pickle('player_id_to_name.pkl')
results_processed = read_pickle('results_processed.pkl')
tournaments_processed = read_pickle('tournaments_processed.pkl')

Для сокращения числа переменных (и чтобы опять же не упереться в ограничения по памяти) оставим только тех игроков, которые играли в 2019 и в 2020 году.

In [5]:
player_ids_set = set()
for tournament in results_processed:
    if tournament['type'] in ['test', 'train']:
        for team_result in tournament['team_results']:
            for player_id in team_result['team_members']:
                player_ids_set.add(player_id)

In [6]:
len(player_ids_set)

61376

In [7]:
player_id_to_idx = {}
player_idx_to_id = []
idx = 0
for player_id in player_ids_set:
    player_id_to_idx[player_id] = idx
    player_idx_to_id.append(player_id)
    idx += 1

In [8]:
players_count = len(player_idx_to_id)

In [9]:
question_to_idx = {}
idx_to_tournament_and_question = {}
idx = players_count
for tournament in results_processed:
    if tournament['type'] == 'train':
        for question_idx in range(len(tournament['team_results'][0]['mask'])):
            question_to_idx[(tournament['tournament_id'], question_idx)] = idx
            idx_to_tournament_and_question[idx] = (tournament['tournament_id'], question_idx)
            idx += 1
vector_length = idx

In [10]:
train_data = []
for tournament in results_processed:
    if tournament['type'] == 'train':
        for team_result in tournament['team_results']:
            for question_idx in range(len(team_result['mask'])):
                data = (tournament['tournament_id'], question_idx, team_result['team_members'], int(team_result['mask'][question_idx]))
                train_data.append(data)

In [11]:
def vectorize_data(data):
    y = []
    row_idx = 0
    row = []
    col = []
    d = []
    for item in data:
        for team_member in item[2]:
            row.append(row_idx)
            col.append(player_id_to_idx[team_member])
            d.append(1.0)
            row.append(row_idx)
            col.append(question_to_idx[(item[0], item[1])])
            d.append(-1.0)
            y.append(item[3])
            row_idx += 1
    x = csr_matrix((np.array(d), (np.array(row), np.array(col))), shape=(row_idx, vector_length))
    
    return x, y

In [12]:
x, y = vectorize_data(train_data)

In [13]:
classifier = LogisticRegression(solver='sag', max_iter=10000)
classifier.fit(x, y)

LogisticRegression(max_iter=10000, solver='sag')

Для того, чтобы предсказать результаты турнира с известными составами команд необходимо оценить силы команд. На этом шаге будем полагать, что сила команды равна среднему арифметическому сил игроков, входящих в неё.

In [14]:
def vectorize_team_members(team_members):
    v = [0] * vector_length
    members_count = len(team_members)
    for team_member in team_members:
        v[player_id_to_idx[team_member]] = 1.0 / members_count
    return v

In [29]:
def get_positions():
    res = []
    for tournament in results_processed:
        if tournament['type'] == 'test':
            positions = []
            positions_pred = []
            position_to_teams = {}
            power_to_teams = {}
            team_to_power = []
            team_to_position = []
            team_idx = 0
            for team_result in tournament['team_results']:
                vectorized_team = vectorize_team_members(team_result['team_members'])
                team_power = np.matmul(vectorized_team, classifier.coef_[0])
                positions_pred.append(-team_power)
                position = team_result['position']
                positions.append(position)
                team_idx += 1
            if len(positions) and len(positions_pred):
                res.append({
                    'positions': positions,
                    'positions_pred': positions_pred
                })
    return res


In [16]:
def get_correlation(positions, corr_type='spearman'):
    corr = []
    for p in positions:
        if corr_type == 'spearman':
            c, _ = stats.spearmanr(p['positions'], p['positions_pred'])
        else:
            c, _ = stats.kendalltau(p['positions'], p['positions_pred'])
        corr.append(c)
    return np.mean(corr)

In [30]:
positions = get_positions()
print('Spearman correlation: ', get_correlation(positions))
print('Kendall correlation: ', get_correlation(positions, 'kendall'))

Spearman correlation:  0.753417091441801
Kendall correlation:  0.5939160097220554


In [31]:
def save_sklearn_model(model, filename):
    joblib.dump(model, filename)
    
def load_sklearn_model(filename):
    return joblib.load(filename)

In [32]:
save_sklearn_model(classifier, 'baseline.pkl')

Фактически baseline модель предсказывает вероятности $p(y_{ij} = 1| z_{ij} = 1, X)$, где i - индекс игрока, j - индекс вопроса, $y_{ij} = 1$ - игрок i правильно ответил на вопрос j, $z_{ij} = 1$ - игрок i отвечал на вопрос j, X - данные.
Но это не совсем то, что нам нужно. Мы хотим найти $p(y_{ij}|X)$:
$$p(y_{ij}|X) = p(y_{ij} = 1| z_{ij} = 1, X)p(z_{ij} = 1| X) + p(y_{ij} = 1| z_{ij} = 0, X)p(z_{ij} = 0| X)$$
Полагаем, что если игрок не отвечал на вопрос, то он и не ответил:
$$p(y_{ij}|X) = p(y_{ij} = 1| z_{ij} = 1, X)p(z_{ij} = 1| X)$$
Интуиция: мы оценивали силу команды как среднее арифметическое сил игроков, можно смотреть на это как на математическое ожидание силы команды при условии равновероятности событий "игрок i отвечал на вопрос j". Зная $p(z_{ij} = 1| X)$ мы могли бы пересчитать математическое ожидание силы команды при условии, что события "игрок i отвечал на вопрос j" на самом деле не равновероятны.
Оценим математическое ожидание силы команды следующим образом:
$${E(w_{A_{i}} | w_j, w_k, k \in A_{i})} = \sum_{B \subset A_{i}} p(B | w_j, w_k, k \in A_{i})(\sum_{r \in B} \frac{w_r}{|B|})$$,
где $A_i$ - множество игроков команды i, $w_k$ - их силы, $w_j$ - сложность вопроса j, $B$ - подмножества игроков команды $A_i$. (Считаем, что сила подмножества игроков команды равна средней силе игроков, входящих в него)

Оценим $p(B | w_j, w_k, k \in A_{i})$ следующим образом:
$$p(B | w_j, w_k, k \in A_{i}) = \prod_{k \in A_{i}} q_{k}^{h_{k}} (1 - q_{k})^{1 - h_{k}}$$, где $q_{k}$ - вероятность, что игрок k может ответить на вопрос j (выход baseline модели), 
$h_{k} = 1$, если $k \in B$,
$h_{k} = 0$, если $k \notin B$

Если привести подобные математическое ожидание силы команды перепишется в виде:
$${E(w_{A_{i}} | w_j, w_k, k \in A_{i})} = \sum_{k \in A_{i}} z_{kj} w_k$$
Правдоподобие:
$$p(y|X, w) = \prod_{ij} \sigma(<(\sum_{k \in A_{i}} z_{kj}v_k) - v_j, w>)^{y_{ij}}(1 - \sigma(<(\sum_{k \in A_{i}} z_{kj}v_k) - v_j, w>))^{1 - y_{ij}}$$, где $v_k$ - one-hot вектор, соответствующий игроку k, $v_j$ - one-hot вектор, соответствующий вопросу j.

На E шаге ЕМ-алгоритма будем пересчитывать $z_{ij}$ исходя из параметров текущей модели, а на М шаге будем переобучать модель с учетом новых значений переменных $z_{ij}$. В качестве нулевого приближения возьмём $z_{ij} = \frac{1}{|A_{i}|}$.

In [33]:
def vectorize_data_zero_approach(data):
    y = []
    row_idx = 0
    row = []
    col = []
    d = []
    for item in data:
        for team_member in item[2]:
            row.append(row_idx)
            col.append(player_id_to_idx[team_member])
            d.append(1.0)
        row.append(row_idx)
        col.append(question_to_idx[(item[0], item[1])])
        d.append(-1.0)
        y.append(item[3])
        row_idx += 1
    x = csr_matrix((np.array(d), (np.array(row), np.array(col))), shape=(row_idx, vector_length))
    
    return x, y

In [34]:
x, y = vectorize_data_zero_approach(train_data)

In [36]:
classifier = LogisticRegression(solver='sag', max_iter=10000, warm_start=True)
classifier.fit(x, y)

LogisticRegression(max_iter=10000, solver='sag', warm_start=True)

In [37]:
positions = get_positions()
print('Spearman correlation: ', get_correlation(positions))
print('Kendall correlation: ', get_correlation(positions, 'kendall'))

Spearman correlation:  0.7532438846063843
Kendall correlation:  0.594136162701013


In [38]:
save_sklearn_model(classifier, 'em_0.pkl')

In [39]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

list(powerset([1, 2, 3]))

[(1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

In [52]:
def get_z_and_vectorize_data(data):
    y = []
    row_idx = 0
    row = []
    col = []
    d = []
    for item in data:
        #team_members_count = len(item[2])
        v = []
        r = []
        c = []
        team_size = len(item[2])
        for team_member_idx in range(team_size):
            r.append(team_member_idx)
            c.append(player_id_to_idx[item[2][team_member_idx]])
            v.append(1.0)
            r.append(team_member_idx)
            c.append(question_to_idx[(item[0], item[1])])
            v.append(-1.0)
        csr = csr_matrix((np.array(v), (np.array(r), np.array(c))), shape=(len(item[2]), vector_length))
        team_member_idx_to_proba = classifier.predict_proba(csr)
        #print(team_member_idx_to_proba)
        team_member_idx_to_z = [0] * team_size
        for subset in powerset(range(team_size)):
            subset_len = len(subset)
            proba = 1.0
            for team_member_idx in range(team_size):
                if team_member_idx in subset:
                    proba *= team_member_idx_to_proba[team_member_idx][1]
                else:
                    proba *= team_member_idx_to_proba[team_member_idx][0]
            for team_member_idx in subset:
                team_member_idx_to_z[team_member_idx] += proba / subset_len
        for team_member_idx in range(team_size):
            row.append(row_idx)
            col.append(player_id_to_idx[item[2][team_member_idx]])
            d.append(team_member_idx_to_z[team_member_idx])
        row.append(row_idx)
        col.append(question_to_idx[(item[0], item[1])])
        d.append(-1.0)
        y.append(item[3])
        row_idx += 1
    x = csr_matrix((np.array(d), (np.array(row), np.array(col))), shape=(row_idx, vector_length))
    
    return x, y

In [54]:
# EM iteration 1
x, y = get_z_and_vectorize_data(train_data)
classifier.fit(x, y)
save_sklearn_model(classifier, 'em_1.pkl')
positions = get_positions()
print('Spearman correlation: ', get_correlation(positions))
print('Kendall correlation: ', get_correlation(positions, 'kendall'))

Spearman correlation:  0.7536896567290119
Kendall correlation:  0.5970675318146211


Качество модели после первой итерации ЕМ алгоритма стало чуть лучше baseline модели. Попробуем сделать ещё несколько итераций

In [55]:
# EM iteration 2
x, y = get_z_and_vectorize_data(train_data)
classifier.fit(x, y)
save_sklearn_model(classifier, 'em_2.pkl')
positions = get_positions()
print('Spearman correlation: ', get_correlation(positions))
print('Kendall correlation: ', get_correlation(positions, 'kendall'))

Spearman correlation:  0.7553983479793784
Kendall correlation:  0.5963987491283853


In [56]:
# EM iteration 3
x, y = get_z_and_vectorize_data(train_data)
classifier.fit(x, y)
save_sklearn_model(classifier, 'em_3.pkl')
positions = get_positions()
print('Spearman correlation: ', get_correlation(positions))
print('Kendall correlation: ', get_correlation(positions, 'kendall'))

Spearman correlation:  0.7562127112516149
Kendall correlation:  0.5975786966159717


Как мы видим целевые метрики с каждой итерацией медленно, но растут.

Построим рейтинг-лист турниров по сложности вопросов. Будем считать, что сложность турнира определяется средней сложностью вопросов.

In [79]:
tournaments_top = []
for tournament in results_processed:
    if tournament['type'] == 'train':
        questions_count = len(tournament['team_results'][0]['mask'])
        difficulties_sum = 0.0
        for question_idx in range(questions_count):
            difficulties_sum += classifier.coef_[0][question_to_idx[(tournament['tournament_id'], question_idx)]]
        difficulty = difficulties_sum / questions_count
        tournaments_top.append((tournament['tournament_id'], tournaments_processed[tournament['tournament_id']]['name'], difficulty))
tournaments_top = sorted(tournaments_top, key=lambda x: -x[2])

Топ 10 турниров:

In [82]:
for x in tournaments_top[:20]:
    print(tournaments_top.index(x), x[1])

0 Чемпионат Санкт-Петербурга. Первая лига
1 Угрюмый Ёрш
2 Первенство правого полушария
3 Воображаемый музей
4 Кубок городов
5 Записки охотника
6 Чемпионат России
7 Знание – Сила VI
8 Ускользающая сова
9 All Cats Are Beautiful
10 Ра-II: синхрон "Борского корабела"
11 VERSUS: Коробейников vs. Матвеев
12 Чемпионат Мира. Этап 2. Группа В
13 Антибинго
14 Чемпионат Мира. Этап 3. Группа В
15 Линч
16 Серия Premier. Седьмая печать
17 Кубок Москвы
18 Тихий Донец: омут первый
19 Львов зимой. Адвокат


Конец списка:

In [84]:
for x in tournaments_top[-20:]:
    print(tournaments_top.index(x), x[1])

592 Осенняя кинолига
593 Лига вузов. IV тур
594 Студенческий чемпионат Калининградской области
595 Межфакультетский кубок МГУ. Отбор №4
596 Лига Сибири. IV тур.
597 KFC
598 Joystick Cup
599 Лига Сибири. VI тур.
600 Гарри Поттер и 3 по 12
601 Синхрон-lite. Выпуск XXIX
602 Второй тематический турнир имени Джоуи Триббиани
603 Школьный Синхрон-lite. Выпуск 2.5
604 (а)Синхрон-lite. Лига старта. Эпизод X
605 Синхрон-lite. Выпуск XXX
606 (а)Синхрон-lite. Лига старта. Эпизод IV
607 (а)Синхрон-lite. Лига старта. Эпизод VI
608 (а)Синхрон-lite. Лига старта. Эпизод VII
609 (а)Синхрон-lite. Лига старта. Эпизод IX
610 (а)Синхрон-lite. Лига старта. Эпизод III
611 (а)Синхрон-lite. Лига старта. Эпизод V


Полученный рейтинг турниров соответствует интуиции: в топ попали турниры, которые должны быть сложными (чемпионаты мира, чемпионат России), в то время как в конец списка попали школьные и студенческие турниры.